In [ ]:
# %conda install --file requirements-poetry.txt
%conda install toml pyarrow jupysql -c conda-forge

: 

In [ ]:
# %pip install 

In [12]:
# download assets from block storage at given URL 
# (our case is zipped folder served through LAN from another machine by miniserve)
%reload_ext dotenv
# %dotenv
import os
from urllib.request import urlretrieve
from pathlib import Path

ASSET_URL, out_file= (os.environ('ASSET_URL') ,'out/testing_data.tar.gz')

if not Path(out_file).is_file():
    os.makedirs(os.path.dirname(out_file), exist_ok=True)
    folder_out, res = urlretrieve(ASSET_URL, filename=out_file)
    print(folder_out)

In [2]:
import tarfile

# Extract files from tar
with tarfile.open(out_file, "r") as file:
    for each in file.getnames():
        print(each)
        file.extract(f'{each}')

data
data/en.openfoodfacts.org.products.csv
data/en.openfoodfacts.org.products-delimited.csv


In [3]:
import polars as pl

df = pl.scan_csv("./data/en.openfoodfacts.org.products-delimited.csv",
                 truncate_ragged_lines=True,
                 )
df.sink_parquet('./data/openfoodfacts_products-delimited.parquet',
                compression="snappy",
                row_group_size=100_000,
                statistics=True,
                )

In [9]:
%reload_ext pyprql.magic
%reload_ext sql

%config SqlMagic.autopolars = True
%config PrqlMagic.autopolars = True
%config SqlMagic.named_parameters=True
# recipes = os.environ['RECIPES_ASSET_URL']
# !wget http://192.168.0.143:3003/recipes.parquet
# !curl -sL https://raw.githubusercontent.com/mwaskom/seaborn-data/master/penguins.csv -o penguins.csv

%prql duckdb:///doto.duckdb
%sql duckdb:///doto.duckdb

# %sql CREATE TABLE off_products AS SELECT * FROM read_parquet('./data/openfoodfacts_products.parquet') LIMIT 200
# df = %sql SELECT * FROM read_parquet('./data/openfoodfacts_products.parquet')


# df = %sql SELECT * FROM test_recipes WHERE REGEXP_MATCHES(name, 'mex*') 
# %sql SELECT n_ingredients FROM read_parquet('http://192.168.0.143:3003/recipes.parquet') WHERE REGEXP_MATCHES(name, 'mex*')
# %sqlplot pie --table test_recipes --column minutes
# TODO: doesn't work w/ polars
# %sqlplot histogram --table test_recipes --column n_ingredients --bins 100

# %sql SELECT * FROM test_recipes WHERE REGEXP_MATCHES(name, 'mex*')

# display(df)

# df = %prql prql target:sql.duckdb
# prql target:sql.duckdb
# from `test_recipes`  
# sort {-n_ingredients} 
# display(df)


Disabled 'autopandas' since 'autopolars' was enabled.

The 'toml' package isn't installed. To load settings from the pyproject.toml file, install with: pip install toml

In [24]:
%%sql 
SELECT * FROM test_recipes 
WHERE REGEXP_MATCHES(name, 'mex*')

Running query in 'duckdb:///doto.duckdb'

_airbyte_ab_id,_airbyte_emitted_at,id,name,tags,index,steps,minutes,n_steps,nutrition,recipe_id,submitted,description,ingredients,created_by_id,n_ingredients,updated_by_id,contributor_id,_airbyte_additional_properties
str,"datetime[μs, Asia/Kuala_Lumpur]",f64,str,str,f64,str,f64,f64,str,f64,struct[2],str,str,f64,f64,f64,f64,list[struct[2]]
"""28c06840-84cf-…",2023-01-04 22:47:23.852 +08,137739.0,"""arriba baked…","""[""60-minutes-o…",0.0,"""[""make a choic…",55.0,11.0,"""[51.5, 0.0, 13…",137739.0,"{2005-09-16 08:00:00 +08,null}","""autumn is my f…","""[""winter squas…",137739.0,7.0,137739.0,47892.0,null
"""a4680eb8-cb12-…",2023-01-04 22:47:23.852 +08,54272.0,"""fool the meat …","""[""60-minutes-o…",26.0,"""[""rehydrate tv…",40.0,9.0,"""[295.6, 3.0, 5…",54272.0,"{2003-02-17 08:00:00 +08,null}","""this recipe wa…","""[""vegetarian g…",54272.0,12.0,54272.0,40525.0,null
"""aabe521a-9c39-…",2023-01-04 22:47:23.852 +08,59952.0,"""global gourmet…","""[""60-minutes-o…",31.0,"""[""heat oven to…",55.0,11.0,"""[456.8, 40.0, …",59952.0,"{2003-04-16 08:00:00 +08,null}","""hey! i found t…","""[""ground beef""…",59952.0,12.0,59952.0,28604.0,null
"""a878d1cd-5203-…",2023-01-04 22:47:23.852 +08,87098.0,"""homemade vege…","""[""15-minutes-o…",35.0,"""[""combine all …",12.0,6.0,"""[276.2, 3.0, 5…",87098.0,"{2004-03-20 08:00:00 +08,null}","""my friend just…","""[""low sodium c…",87098.0,8.0,87098.0,70662.0,null
"""5eb6dbe2-48ca-…",2023-01-04 22:47:23.852 +08,90921.0,"""i stole the id…","""[""30-minutes-o…",40.0,"""[""in a large p…",18.0,8.0,"""[783.4, 46.0, …",90921.0,"{2004-05-10 08:00:00 +08,null}","""i love mirj's …","""[""angel hair p…",90921.0,8.0,90921.0,15851.0,null
"""f3678ddc-973b-…",2023-01-04 22:47:23.852 +08,58224.0,"""immoral sandw…","""[""60-minutes-o…",43.0,"""[""brown the me…",35.0,6.0,"""[223.2, 22.0, …",58224.0,"{2003-04-04 08:00:00 +08,null}","""just the thing…","""[""ground beef""…",58224.0,8.0,58224.0,37183.0,null
"""5afed554-918d-…",2023-01-04 22:47:23.852 +08,44045.0,"""mennonite cor…","""[""15-minutes-o…",59.0,"""[""combine eggs…",15.0,6.0,"""[67.1, 7.0, 2.…",44045.0,"{2002-10-25 08:00:00 +08,null}","""ok - my herita…","""[""fresh corn"",…",44045.0,8.0,44045.0,41706.0,null
"""a1496daa-8342-…",2023-01-04 22:47:23.852 +08,60219.0,"""mexican pasta""","""[""30-minutes-o…",60.0,"""[""boil pasta a…",25.0,5.0,"""[325.3, 10.0, …",60219.0,"{2003-04-18 08:00:00 +08,null}","""made this as a…","""[""fettuccine p…",60219.0,6.0,60219.0,76846.0,null
"""ac48b566-8bfe-…",2023-01-04 22:47:23.852 +08,107229.0,"""open sesame n…","""[""30-minutes-o…",73.0,"""[""cook pasta i…",28.0,8.0,"""[617.5, 34.0, …",107229.0,"{2004-12-30 08:00:00 +08,null}","""this is a very…","""[""vermicelli"",…",107229.0,12.0,107229.0,173674.0,null


In [26]:
%%prql
prql target:sql.duckdb

from `test_recipes`  
filter (name ~= "mexican")
sort {-n_ingredients} 
take 5..25

_airbyte_ab_id,_airbyte_emitted_at,id,name,tags,index,steps,minutes,n_steps,nutrition,recipe_id,submitted,description,ingredients,created_by_id,n_ingredients,updated_by_id,contributor_id,_airbyte_additional_properties
str,"datetime[μs, Asia/Kuala_Lumpur]",f64,str,str,f64,str,f64,f64,str,f64,struct[2],str,str,f64,f64,f64,f64,list[struct[2]]
"""a61c66a9-adaf-…",2023-01-04 22:47:23.852 +08,24441.0,"""mexican chicke…","""[""weeknight"", …",134046.0,"""[""saut the chi…",330.0,10.0,"""[453.1, 14.0, …",24441.0,"{2002-04-08 08:00:00 +08,null}","""this is top se…","""[""chicken brea…",24441.0,24.0,24441.0,30716.0,null
"""37fc9d4e-fbec-…",2023-01-04 22:47:23.852 +08,352776.0,"""easy mexican t…","""[""60-minutes-o…",77143.0,"""[""combine the …",40.0,7.0,"""[209.2, 5.0, 4…",352776.0,"{2009-01-29 08:00:00 +08,null}","""this is an eas…","""[""whole wheat …",352776.0,24.0,352776.0,754635.0,null
"""1d111500-14d8-…",2023-01-04 22:47:23.852 +08,114910.0,"""mexican style …","""[""30-minutes-o…",134728.0,"""[""fry the baco…",30.0,8.0,"""[445.7, 58.0, …",114910.0,"{2005-04-01 08:00:00 +08,null}","""here's my take…","""[""boneless ski…",114910.0,23.0,114910.0,39547.0,null
"""1193cd1f-52dd-…",2023-01-04 22:47:23.852 +08,490731.0,"""blackened mexi…","""[""30-minutes-o…",24975.0,"""[""to prepare b…",30.0,13.0,"""[767.5, 88.0, …",490731.0,"{2012-11-21 08:00:00 +08,null}","""something new …","""[""chicken brea…",490731.0,23.0,490731.0,1.072593e6,null
"""16b9a4ee-2ac5-…",2023-01-04 22:47:23.852 +08,49084.0,"""mexican cheese…","""[""time-to-make…",134022.0,"""[""heat oven to…",420.0,15.0,"""[324.5, 40.0, …",49084.0,"{2002-12-17 08:00:00 +08,null}","""this makes for…","""[""tortilla chi…",49084.0,22.0,49084.0,44689.0,null
"""f3f3474d-c533-…",2023-01-04 22:47:23.852 +08,222017.0,"""mexican spicy …","""[""time-to-make…",134675.0,"""[""preheat oven…",70.0,24.0,"""[1133.4, 64.0,…",222017.0,"{2007-04-11 08:00:00 +08,null}","""from south ame…","""[""onion"", ""gar…",222017.0,22.0,222017.0,353579.0,null
"""ddc4bfcc-817a-…",2023-01-04 22:47:23.852 +08,486261.0,"""mexican stack …","""[""time-to-make…",134684.0,"""[""preheat broi…",75.0,28.0,"""[793.0, 83.0, …",486261.0,"{2012-09-01 08:00:00 +08,null}","""ready, set, co…","""[""poblano pepp…",486261.0,22.0,486261.0,226863.0,null
"""fc5b45d6-53b4-…",2023-01-04 22:47:23.852 +08,140012.0,"""mexican street…","""[""time-to-make…",134695.0,"""[""to make the …",105.0,18.0,"""[989.3, 95.0, …",140012.0,"{2005-10-04 08:00:00 +08,null}","""from the tony …","""[""flank steaks…",140012.0,22.0,140012.0,120566.0,null
"""28e9a421-1975-…",2023-01-04 22:47:23.852 +08,365027.0,"""pinto bean chi…","""[""course"", ""ma…",160804.0,"""[""in a large s…",750.0,15.0,"""[486.1, 28.0, …",365027.0,"{2009-04-08 08:00:00 +08,null}","""beth hensperge…","""[""olive oil"", …",365027.0,22.0,365027.0,37779.0,null
